Goal: Clean the three datasets scraped from [this website](https://howlongtobeat.com). The script got the playtime information of about 50000 games on different platforms, in addition to information about how many people had included their input for that game. 

In [33]:
import json
import pandas as pd
import numpy as np
import re
import math

In [34]:
game_times = pd.read_json('./data/time.json')

View the columns and entries for each datasets

In [35]:
game_times

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,A Plague Tale: Innocence,game?id=65080,10½ Hours,search_list_tidbit center time_100,12½ Hours,search_list_tidbit center time_100,16 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
1,The Witcher 3: Wild Hunt,game?id=10270,51 Hours,search_list_tidbit center time_100,102 Hours,search_list_tidbit center time_100,172 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
2,The Legend of Zelda: Skyward Sword HD,game?id=88748,29½ Hours,search_list_tidbit center time_100,35 Hours,search_list_tidbit center time_100,37 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
3,Tomb Raider,game?id=10469,11½ Hours,search_list_tidbit center time_100,15½ Hours,search_list_tidbit center time_100,20½ Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
4,Hollow Knight,game?id=26286,26 Hours,search_list_tidbit center time_100,39½ Hours,search_list_tidbit center time_100,57½ Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48903,Damnatio Memoriae,game?id=73404,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48904,The Wizard Sniffer,game?id=73406,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48905,Suveh Nux,game?id=73409,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48906,Defenders of Tetsoidea,game?id=73481,4½ Hours,search_list_tidbit center time_40,--,search_list_tidbit center time_00,--,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
game_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48908 entries, 0 to 48907
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    48908 non-null  object
 1   url                     48908 non-null  object
 2   Main Story time         45865 non-null  object
 3   Main Story tag info     45865 non-null  object
 4   Main + Extra time       45865 non-null  object
 5   Main + Extra tag info   45865 non-null  object
 6   Completionist time      45865 non-null  object
 7   Completionist tag info  45865 non-null  object
 8   Solo time               1959 non-null   object
 9   Solo tag info           1959 non-null   object
 10  Co-Op time              1867 non-null   object
 11  Co-Op tag info          1867 non-null   object
 12  Vs. time                2482 non-null   object
 13  Vs. tag info            2482 non-null   object
dtypes: object(14)
memory usage: 5.2+ MB


On their website, information that isn't available about the completion time of something is shown by the character **--**. For each entry, we can check and replace this character with Nan values.

In [37]:
def replace_with_null(x):
    if type(x) == str:
        nospace_x = x.replace(' ', '').strip()
    else:
        return x
    if nospace_x == '--':
        return np.nan
    else:
        return x

In [38]:
game_times = game_times.applymap(replace_with_null)

In [39]:
game_times

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,A Plague Tale: Innocence,game?id=65080,10½ Hours,search_list_tidbit center time_100,12½ Hours,search_list_tidbit center time_100,16 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
1,The Witcher 3: Wild Hunt,game?id=10270,51 Hours,search_list_tidbit center time_100,102 Hours,search_list_tidbit center time_100,172 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
2,The Legend of Zelda: Skyward Sword HD,game?id=88748,29½ Hours,search_list_tidbit center time_100,35 Hours,search_list_tidbit center time_100,37 Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
3,Tomb Raider,game?id=10469,11½ Hours,search_list_tidbit center time_100,15½ Hours,search_list_tidbit center time_100,20½ Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
4,Hollow Knight,game?id=26286,26 Hours,search_list_tidbit center time_100,39½ Hours,search_list_tidbit center time_100,57½ Hours,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48903,Damnatio Memoriae,game?id=73404,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48904,The Wizard Sniffer,game?id=73406,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48905,Suveh Nux,game?id=73409,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48906,Defenders of Tetsoidea,game?id=73481,4½ Hours,search_list_tidbit center time_40,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN


For the columns that represent time, remove Hours from them. If there is a fraction sign remove it, turn str to int and thn add 0.5 to it.

In [40]:
def exrtact_num_from_txt(txt):
    txt = txt.strip()
    num = ''
    for c in txt:
        if c in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
            num = num + c
    return int(num)

In [41]:
def time_to_int(x):
    if x is not np.nan:
        num = ''
        has_half = False
        if '½' in x:
            has_half = True
        num = exrtact_num_from_txt(x)
        if has_half:
            num += 0.5
        return num
    else:
        return x

In [42]:
#All columns except name and url
time_cols = []
for c in game_times.columns:
    if 'time' in c:
        time_cols.append(c)

#Change the format of all columns that have data about time to floats
game_times[time_cols] = game_times[time_cols].applymap(time_to_int)

In [43]:
game_times

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,A Plague Tale: Innocence,game?id=65080,10.5,search_list_tidbit center time_100,12.5,search_list_tidbit center time_100,16.0,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
1,The Witcher 3: Wild Hunt,game?id=10270,51.0,search_list_tidbit center time_100,102.0,search_list_tidbit center time_100,172.0,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
2,The Legend of Zelda: Skyward Sword HD,game?id=88748,29.5,search_list_tidbit center time_100,35.0,search_list_tidbit center time_100,37.0,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
3,Tomb Raider,game?id=10469,11.5,search_list_tidbit center time_100,15.5,search_list_tidbit center time_100,20.5,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
4,Hollow Knight,game?id=26286,26.0,search_list_tidbit center time_100,39.5,search_list_tidbit center time_100,57.5,search_list_tidbit center time_100,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48903,Damnatio Memoriae,game?id=73404,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48904,The Wizard Sniffer,game?id=73406,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48905,Suveh Nux,game?id=73409,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN
48906,Defenders of Tetsoidea,game?id=73481,4.5,search_list_tidbit center time_40,NaN,search_list_tidbit center time_00,NaN,search_list_tidbit center time_00,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
tag_info_cols = []
for col in game_times.columns:
    if 'tag' in col:
        tag_info_cols.append(col)


#replace with a lambda functions
def temp(entry):
    if entry is not np.nan:
        return exrtact_num_from_txt(entry)
    else:
        return np.nan
    
game_times[tag_info_cols] = game_times[tag_info_cols].applymap(temp)

In [45]:
game_times

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,A Plague Tale: Innocence,game?id=65080,10.5,100.0,12.5,100.0,16.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
1,The Witcher 3: Wild Hunt,game?id=10270,51.0,100.0,102.0,100.0,172.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
2,The Legend of Zelda: Skyward Sword HD,game?id=88748,29.5,100.0,35.0,100.0,37.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Tomb Raider,game?id=10469,11.5,100.0,15.5,100.0,20.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Hollow Knight,game?id=26286,26.0,100.0,39.5,100.0,57.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48903,Damnatio Memoriae,game?id=73404,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48904,The Wizard Sniffer,game?id=73406,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48905,Suveh Nux,game?id=73409,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48906,Defenders of Tetsoidea,game?id=73481,4.5,40.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Next step is to remove the Spacing between all names to make it ready.

In [46]:
def clear(string):
    """return: Returns the string, stripped of all letters except alphanum
    If the string doesnt contain alphanum or only conatins numbers, None is returned"""
    string = (re.sub("[^A-Za-z0-9]+", "", string)).lower()
    if (string.isdecimal()) or (len(string) == 0):
        return np.nan
    return string

In [47]:
game_times["name"] = game_times["name"].apply(clear)
game_times

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,aplaguetaleinnocence,game?id=65080,10.5,100.0,12.5,100.0,16.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
1,thewitcher3wildhunt,game?id=10270,51.0,100.0,102.0,100.0,172.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
2,thelegendofzeldaskywardswordhd,game?id=88748,29.5,100.0,35.0,100.0,37.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,tombraider,game?id=10469,11.5,100.0,15.5,100.0,20.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
4,hollowknight,game?id=26286,26.0,100.0,39.5,100.0,57.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48903,damnatiomemoriae,game?id=73404,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48904,thewizardsniffer,game?id=73406,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48905,suvehnux,game?id=73409,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48906,defendersoftetsoidea,game?id=73481,4.5,40.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
game_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48908 entries, 0 to 48907
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    48878 non-null  object 
 1   url                     48908 non-null  object 
 2   Main Story time         24172 non-null  float64
 3   Main Story tag info     45865 non-null  float64
 4   Main + Extra time       16363 non-null  float64
 5   Main + Extra tag info   45865 non-null  float64
 6   Completionist time      18557 non-null  float64
 7   Completionist tag info  45865 non-null  float64
 8   Solo time               1125 non-null   float64
 9   Solo tag info           1959 non-null   float64
 10  Co-Op time              267 non-null    float64
 11  Co-Op tag info          1867 non-null   float64
 12  Vs. time                411 non-null    float64
 13  Vs. tag info            2482 non-null   float64
dtypes: float64(12), object(2)
memory usage

In [49]:
game_times = game_times[game_times['name'].notna()]

In [50]:
to_drop = []
times = [col for col in game_times.columns if "time" in col]
for i, row in game_times.iterrows():
        curr_row_times = 0
        for time_col in times:
            if not math.isnan(row[time_col]):
                curr_row_times =+ 1 
        if curr_row_times == 0:
            to_drop.append(i)

In [51]:
to_drop

[285,
 798,
 823,
 831,
 923,
 948,
 1030,
 1229,
 1258,
 1267,
 1284,
 1529,
 1600,
 1615,
 1657,
 1692,
 1739,
 1745,
 1761,
 1785,
 1805,
 1873,
 1895,
 1898,
 1954,
 1955,
 1959,
 1977,
 1998,
 2021,
 2022,
 2046,
 2071,
 2125,
 2140,
 2168,
 2181,
 2251,
 2258,
 2264,
 2287,
 2295,
 2313,
 2339,
 2408,
 2413,
 2416,
 2484,
 2570,
 2604,
 2632,
 2679,
 2710,
 2728,
 2763,
 2861,
 2868,
 2942,
 2989,
 2993,
 3011,
 3079,
 3088,
 3172,
 3178,
 3187,
 3195,
 3216,
 3229,
 3235,
 3265,
 3266,
 3295,
 3322,
 3331,
 3334,
 3400,
 3462,
 3496,
 3518,
 3547,
 3551,
 3579,
 3601,
 3602,
 3603,
 3610,
 3655,
 3659,
 3660,
 3663,
 3726,
 3760,
 3776,
 3843,
 3845,
 3846,
 3885,
 3998,
 4002,
 4013,
 4045,
 4088,
 4092,
 4099,
 4127,
 4136,
 4173,
 4195,
 4202,
 4206,
 4209,
 4213,
 4216,
 4268,
 4273,
 4277,
 4285,
 4296,
 4303,
 4306,
 4312,
 4354,
 4362,
 4369,
 4371,
 4382,
 4387,
 4395,
 4409,
 4443,
 4451,
 4537,
 4550,
 4569,
 4573,
 4648,
 4651,
 4660,
 4662,
 4663,
 4672,
 4673,
 4684

In [55]:
game_timescc = game_times.drop(labels=to_drop, axis=0)

In [56]:
game_timescc

,name,url,Main Story time,Main Story tag info,Main + Extra time,Main + Extra tag info,Completionist time,Completionist tag info,Solo time,Solo tag info,Co-Op time,Co-Op tag info,Vs. time,Vs. tag info
0,aplaguetaleinnocence,game?id=65080,10.5,100.0,12.5,100.0,16.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
1,thewitcher3wildhunt,game?id=10270,51.0,100.0,102.0,100.0,172.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
2,thelegendofzeldaskywardswordhd,game?id=88748,29.5,100.0,35.0,100.0,37.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,tombraider,game?id=10469,11.5,100.0,15.5,100.0,20.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
4,hollowknight,game?id=26286,26.0,100.0,39.5,100.0,57.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48891,coloringpixelsisometric,game?id=73320,NaN,0.0,NaN,0.0,30.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
48892,santagirls,game?id=73328,3.5,40.0,NaN,0.0,8.5,40.0,NaN,NaN,NaN,NaN,NaN,NaN
48898,thehurricaneofthevarstraythreatofthirdforce,game?id=73382,15.0,40.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48900,unsolvedmysteryclubancientastronauts,game?id=73387,NaN,0.0,NaN,0.0,3.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
game_timescc.to_csv('timeclean.csv')